# Project

In [2]:
import time
import os
import pprint
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import librosa
import librosa.display
import utils

In [3]:
AUDIO_DIR = os.environ.get('AUDIO_DIR')
print("audio directory: ",AUDIO_DIR)
print("Loading tracks.csv...")
tracks = utils.load('data/fma_metadata/tracks.csv')
#get only the small subset of the dataset
small = tracks[tracks['set', 'subset'] <= 'small']
print("small dataset shape:",small.shape)

audio directory:  ./data/fma_small/
Loading tracks.csv...
small dataset shape: (8000, 52)


In [4]:
#split into train, validation and test sets
train = small.index[small['set', 'split'] == 'training']
val = small.index[small['set', 'split'] == 'validation']
test = small.index[small['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))


6400 training examples, 800 validation examples, 800 testing examples


In [5]:
genres=utils.load('data/fma_metadata/genres.csv')
#list of genres
genres_list=genres['title'].unique()

In [6]:
def expand_labels(labels_set):
    lbl_array = labels_set

    # Repeat each element 10 times using np.repeat()
    expanded_array = np.repeat(lbl_array, 10)

    # Create a new Categorical object from the expanded array
    expanded_categorical = pd.Categorical(expanded_array)

    # Print the expanded categorical variable
    return expanded_categorical

In [1]:
#retrieve labels for each subset
tr_labels = small.loc[small[('set', 'split')] == 'training', ('track', 'genre_top')].values
vl_labels = small.loc[small[('set', 'split')] == 'validation', ('track', 'genre_top')].values
ts_labels = small.loc[small[('set', 'split')] == 'test', ('track', 'genre_top')].values

tr_labels=expand_labels(tr_labels)
vl_labels=expand_labels(vl_labels)
ts_labels=expand_labels(ts_labels)


print('{} training labels, {} validation labels, {} testing labels'.format(*map(len, [tr_labels, vl_labels, ts_labels])))


NameError: name 'small' is not defined

## Generate the STFT small dataset

### Let's get all the STFT spectrograms for each track audio in the dataset

Some tracks of the original small dataset are corrupted or very small (1-2 seconds), these track will be replaced
by a script below with other good tracks of the same genre from the dataset.

#### Important
Create a directory 'fma_small_stft' with 3 directories inside:
- 'train'
- 'validation'
- 'test'

In [ ]:
def generate_stft_files(dataset,directory):
    print("Generating files in the folder:",directory,"...")

    track_ids = np.array(dataset)
    
    error_list = [] #list of tracks which caused errors
    progress_bar = 1
    for track_id in track_ids:
        print("track id: ",track_id)

        try:
            #get the filename using the utils
            filename = utils.get_audio_path(AUDIO_DIR, track_id)
            print('File: {}'.format(filename))

            x, sr = librosa.load(filename, sr=None, mono=True) #load the MONO audio file from the data/fma_small directory
            print('Duration: {:.2f}s, {} samples'.format(x.shape[-1] / sr, x.size))

            #calculate stft on each 3s clip of the 30s song
            for i in range (0,28,3):
                start, end = i, i+3 #start and end point of the clip
                clip = x[start*sr:end*sr] #extract the clip
                #calculate stft using a window of 1024 sample and a hop length of 512 sample
                stft = np.abs(librosa.stft(clip, n_fft=1024, hop_length=512))
                #resize the spectrogram to become 512 x 128 instead of 512 x 130 (or other shapes)
                stft = librosa.util.fix_length(stft, size=513, axis=0)[:, :128]

                save_filename = './data/fma_small_stft/' + directory + "/" +  str(track_id) + "_" + str(i//3)
                #print("Saving the stft vector in file:",save_filename) 
                np.save(save_filename,stft)
                #print("shape of the stft vector (clip " + str(i//3) + "): ", stft.shape)
                
        except Exception as e: #skip the songs which give error (file corrupted or song too short)
            print("ERROR on file: ",filename,"\nError information:", e)
            error_list.append(filename)
            
        print("Progress:" + str(progress_bar) + "/" + str(len(track_ids)))
        progress_bar +=1
        if(len(error_list)>0):
            print("\n***Some tracks caused errors: ", error_list)
        else:
            print("\n***Procedure completed without errors.")

In [ ]:
generate_stft_files(train,'train')

In [ ]:
generate_stft_files(val,'validation')

In [ ]:
generate_stft_files(test,'test')